In [ ]:
#SROIE2019 데이터를 UFO형식으로 변환
#https://www.kaggle.com/datasets/urbikn/sroie-datasetv2

import os
import json
from datetime import datetime

def create_json_from_box_files(box_folder, output_json_path):
    data = {"images": {}}
    
    # 모든 텍스트 파일을 읽어서 JSON 데이터로 변환
    for filename in os.listdir(box_folder):
        if filename.endswith(".txt"):
            # 이미지 파일 이름 설정
            image_id = filename.replace(".txt", ".jpg")
            file_path = os.path.join(box_folder, filename)
            
            # 필요한 정보를 추출하여 새로운 데이터 구조로 변환
            words = {}
            idx = 1
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                for line_num, line in enumerate(f, start=1):
                    # 빈 줄인지 확인
                    if not line.strip():
                        print(f"{filename}의 {line_num}번째 줄이 비어 있습니다.")
                        continue  # 빈 줄 건너뛰기
                    
                    # 좌표와 텍스트 추출
                    parts = [part.strip() for part in line.strip().split(',')]
                    
                    # 좌표가 8개 이상인지 확인
                    if len(parts) < 8:
                        print(f"{filename}의 {line_num}번째 줄에서 좌표 데이터가 부족합니다: {line}")
                        continue  # 데이터가 부족한 줄 건너뛰기
                    
                    try:
                        # 좌표를 정수로 변환
                        x1, y1, x2, y2, x3, y3, x4, y4 = map(int, parts[:8])
                        text = ','.join(parts[8:])  # 나머지 부분을 텍스트로 결합
                    except ValueError as e:
                        print(f"{filename}의 {line_num}번째 줄에서 좌표 변환 중 오류가 발생했습니다: {line} - 오류: {e}")
                        continue  # 오류 발생 시 해당 줄 건너뛰기

                    # 단어 정보 추가
                    words[f"{idx:04d}"] = {
                        "transcription": text,
                        "points": [
                            [x1, y1],
                            [x2, y2],
                            [x3, y3],
                            [x4, y4]
                        ]
                    }
                    idx += 1

            # JSON 구조 생성
            data["images"][image_id] = {
                "paragraphs": {},
                "words": words,
                "chars": {},
                "img_w": 0,  # 이미지의 폭, 필요하면 설정
                "img_h": 0,  # 이미지의 높이, 필요하면 설정
                "num_patches": None,
                "tags": [],
                "relations": {},
                "annotation_log": {
                    "worker": "worker",
                    "timestamp": datetime.now().strftime("%Y-%m-%d"),
                    "tool_version": "",
                    "source": None
                },
                "license_tag": {
                    "usability": True,
                    "public": False,
                    "commercial": True,
                    "type": None,
                    "holder": "Upstage"
                }
            }
    
    # 정리된 데이터 저장
    with open(output_json_path, "w") as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)

# 경로 설정
box_folder = "/data/ephemeral/home/MCG/SROIE2019/train/box"  # 텍스트 박스 파일들이 있는 폴더 경로
output_json_path = "/data/ephemeral/home/MCG/SROIE2019/train/annotations.json"  # 출력 JSON 파일 경로

# 함수 실행
create_json_from_box_files(box_folder, output_json_path)
